<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Stochastic-Optimization" data-toc-modified-id="Stochastic-Optimization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Stochastic Optimization</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Data-Imports" data-toc-modified-id="Data-Imports-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Data Imports</a></span></li><li><span><a href="#Functions-for-Variables-and-Constraints" data-toc-modified-id="Functions-for-Variables-and-Constraints-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Functions for Variables and Constraints</a></span></li><li><span><a href="#Model-Creation" data-toc-modified-id="Model-Creation-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Model Creation</a></span></li></ul></li></ul></li><li><span><a href="#Robust-Optimization" data-toc-modified-id="Robust-Optimization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Robust Optimization</a></span></li><li><span><a href="#Benders-Decomposition-with-Stochastic-programming" data-toc-modified-id="Benders-Decomposition-with-Stochastic-programming-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Benders Decomposition with Stochastic programming</a></span></li></ul></div>

# Stochastic Optimization


Stochastic optimization model used to solve the two-stage problem

In [1]:
using JuMP,HiGHS,DataFrames, GLPK
using CSV
using DataFrames

### Data Imports

In [2]:
dateiStart = "1_buses_"

# Import data about buses
df_buses = CSV.read(string("data/",dateiStart,"paperdata.csv"), DataFrame,types=Dict("C" => String,"I" => String, "C_cur" => String, "Lambda" => String))
#Import demand data
df_demands = CSV.read(string("data/",dateiStart,"demand.csv"), DataFrame)
# Import the realized wind energy
#df_realized_energy = CSV.read(string("data/",dateiStart,"realizedEnergy.csv"), DataFrame)
# Import the remaining data
df_other = CSV.read(string("data/",dateiStart,"other_data.csv"), DataFrame)
# Import the realized wind energy
df_realized_energy_scenarios = CSV.read(string("data/15_zones_scenarios.csv"), DataFrame)

Row,Node,Q,Omega,Energy
,Int64,Int64,Int64,String
1,1,1,1,0.1424043634282774;0.144210969386013;0.11988087544620207;0.11671758859344804;0.11960952246048036;0.12482732392742532;0.12715439461913772;0.11961921410390236;0.13055074038928308;0.11931038251089561;0.12722993431488186;0.16117609798741356;0.18277840333637954;0.198588648153477;0.20999632645455624;0.23693158274963935;0.23402132344241613;0.2168685678190485;0.2156496868123431;0.21487491527510108;0.19692054015303043;0.17507165835148314;0.15774903513640248;0.1536693664338863
2,1,1,2,0.24925152559364394;0.23660657293347845;0.2547726078975398;0.2652790149419735;0.2666329641104493;0.2647527310067237;0.28023064024832584;0.29483431221458467;0.30195889560570166;0.31900485217471775;0.3110440256075149;0.3046755155103205;0.2973436978306434;0.2731402298296725;0.2514808375322323;0.22248500814580463;0.18812457313677608;0.17922818173641133;0.14537280849930967;0.12966009374807386;0.1228492341775822;0.11352103053779045;0.11219749225114242;0.11694216466549477
3,1,1,3,0.38113445680697255;0.39934607673258554;0.4095107449387022;0.4199402307948202;0.4635363363600875;0.4838429672761221;0.45815356128816886;0.45962077955999875;0.4608940710033462;0.46237447610888494;0.4735992856532162;0.47453501395669934;0.491932889023617;0.5217230384668552;0.5377119579466558;0.5354619885980622;0.5443787862519629;0.5432984670882294;0.5129083261134971;0.5315965861615052;0.5214812424082307;0.5057010037234905;0.49970110245781685;0.4519300793932842
4,1,1,4,0.3562393871699561;0.34706232693889427;0.3532945402036468;0.3687214774982786;0.35007255108896757;0.3502445038004581;0.3484569469061921;0.33878830001140076;0.3438864381025607;0.35772195470643675;0.35589277676492775;0.357784382542768;0.3588229668138723;0.36820877099069554;0.34497861590284434;0.3374650414691527;0.3201062193718641;0.3235007130748609;0.3337909840791105;0.301732519199888;0.2703964802160043;0.24991415635967834;0.2372936232219796;0.23129461265265638
5,1,1,5,0.3177469258584971;0.3008752289768007;0.29834936539598067;0.29615703918518377;0.28857681157250786;0.2819413650761805;0.266231092998373;0.24709347854298014;0.21962456636810757;0.19789840059008557;0.16359869712043917;0.14625633643043243;0.13622391087882296;0.12193326350196675;0.11342517260144637;0.10470287732420024;0.10833499827307795;0.09532711402325825;0.09732845217421128;0.09208974391807369;0.09512147179098973;0.11708501191761549;0.13257660000960908;0.13745296104775367
6,1,1,6,0.15435208076029977;0.12113098001343195;0.09788173718030112;0.0804328875933887;0.07333247529883019;0.08343308031208146;0.07022760509455989;0.07094668164071445;0.08041706269205363;0.07590600684310625;0.08122946010385991;0.09874598762959325;0.10682768353886044;0.09671283194925716;0.07510856005091932;0.07300849647141684;0.07514930402562282;0.07134691740407961;0.08622084885543177;0.0868854578153834;0.09390762986567582;0.10197668061054715;0.0932664114660432;0.09884809643701631
7,1,1,7,0.21828681813942943;0.21515993483004814;0.20767110010063328;0.1993867294992019;0.18353497909808816;0.1734698614029331;0.17292600677572303;0.17881719196379514;0.19093561011171012;0.19228161887711007;0.18288245115593735;0.19570253710992702;0.19275894564364662;0.18051490786014765;0.18200145725032488;0.19751892864922307;0.20982077916088915;0.22134182252453458;0.23930782615262358;0.2590937895924542;0.27382928991344613;0.30202949779754606;0.3277103985144704;0.32992782961622075
8,1,1,8,0.25862492418151733;0.25975650254816807;0.24953751035224667;0.23068437678046524;0.2031330752775938;0.1766320865840305;0.13413771756306855;0.10186319657323725;0.07619941396756216;0.05355012904949481;0.0553493166326664;0.061423000891508685;0.061145140086509925;0.058198814145218325;0.06644235726996184;0.08028433959449277;0.09008971066994341;0.10236930231901496;0.10750230455808549;0.09510652694450542;0.10787319529756394;0.11167002300560692;0.12410473262649876;0.13724691845094805
9,1,1,9,0.17621019591993808;0.16972795408279373;0.16349928548822829;0.1764269595148843;0.21129128126853516;0.23

In [3]:
"""
Transforms indices into range for lists of objects (such as generation units)
Input of 0 or empty cells is translated to no objects (i.e. empty array)
"""
function string_to_range(s)
    # Check if s is missing or a string that represents an empty cell
    if s == "missing" || s == "0"
        return Int[]  # Return an empty array of Int type
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by comma
        nums = parse.(Int, nums)  # Parse each part to an integer
        return minimum(nums):maximum(nums)  # Return the range
    end
end

"""
Transforms indices into array of values (such as generation costs per unit for each generation unit)
Input of missing cells is translated to no values (i.e. empty array)
Input of 0 is translated to 0 since it can be a coefficient
"""
function string_to_array(s)
    # Check if s is missing or a string that represents an empty cell
    #if s == "0" return 0  # Return an empty array of Float64 type

    if s == "missing"
        return Float64[]
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by semicolon
        nums = parse.(Float64, nums)  # Parse each part to a float
        return nums  # Return the array of floats
    end
end
"""
Transforms a range and an array into one dictionary
"""
function array_to_dict(keys::AbstractVector{T}, values::AbstractVector{S}) where {T,S}
    # Check if arrays have identical length
    if length(keys) != length(values)
        throw(ArgumentError("Array length must be identical"))
    end

    # Initialisierung des leeren Dictionary
    result_dict = Dict{T,S}()

    # Hinzufügen von Schlüssel-Wert-Paaren zum Dictionary
    for i in 1:length(keys)
        result_dict[keys[i]] = values[i]
    end

    return result_dict
end


array_to_dict

In [4]:
# Create an empty dictionary to hold all nodes
data = Dict()

# Iterate over the range of nodes
for singleNode in eachrow(df_buses)
    # Create a dictionary for the current node with all other variables
    data_node = Dict(
        :I => string_to_array(string.(singleNode.I)),
        :Q => string_to_array(string.(singleNode.Q)),
        :J => string_to_array(string.(singleNode.J)),
        :Λ => string_to_array(string.(singleNode.Lambda)),
        :C_save => string_to_array(string(singleNode.C)) ,
        :C_RU_save => string_to_array(string(singleNode.C_RU)),
        :C_RD_save => string_to_array(string(singleNode.C_RD)),
        :C_U_save => string_to_array(string(singleNode.C_U)),
        :C_D_save => string_to_array(string(singleNode.C_D)),
        :V_LOL_save => string_to_array(string(singleNode.V_LOL)),
        :C_cur_save => string_to_array(string(singleNode.C_cur)),
        :P_max_save => string_to_array(string(singleNode.P_max)),
        :b_save => string_to_array(string(singleNode.b)),
        :LC_Max_save => string_to_array(string(singleNode.LC_Max)),
        :W_s_save => string_to_array(string(singleNode.W_s)),
        :W_d_max_save => string_to_array(string(singleNode.W_d_max)),
        :W_s => Dict(),
        :W_d_min => Dict(),
        :W_d_max => Dict(),
        :L_save => Dict(),
        :C => Dict(),
        :C_RU => Dict(),
        :C_RD => Dict(),
        :C_U => Dict(),
        :C_D => Dict(),
        :P_max => Dict(),
        :C_cur => Dict(),
        :b => Dict(),
        :LC_Max => Dict(),
        :V_LOL => Dict(),
        :Ω => 0,
        :T => 1:1,
    )    
    # Assign the dictionary to the current node key
    data[singleNode.Node] = data_node
end

println(data[2][:Q])
#print(data[1][:W_s])


[2.0]


In [5]:
print(data[2][:Q][1])

2.0

In [6]:
for singleNode in keys(data)
    # Create a dictionary for the current node with all other variables
    data[singleNode][:C] = array_to_dict(data[singleNode][:I], data[singleNode][:C_save])
    data[singleNode][:C_RU] = array_to_dict(data[singleNode][:I], data[singleNode][:C_RU_save])
    data[singleNode][:C_RD] = array_to_dict(data[singleNode][:I], data[singleNode][:C_RD_save])
    data[singleNode][:C_U] = array_to_dict(data[singleNode][:I], data[singleNode][:C_U_save])
    data[singleNode][:C_D] = array_to_dict(data[singleNode][:I], data[singleNode][:C_D_save])
    data[singleNode][:P_max] = array_to_dict(data[singleNode][:I], data[singleNode][:P_max_save])
    data[singleNode][:C_cur] = array_to_dict(data[singleNode][:Q], data[singleNode][:C_cur_save])
    data[singleNode][:b] = array_to_dict(data[singleNode][:Λ], data[singleNode][:b_save])
    data[singleNode][:LC_Max] = array_to_dict(data[singleNode][:Λ], data[singleNode][:LC_Max_save])
    data[singleNode][:V_LOL] = array_to_dict(data[singleNode][:J], data[singleNode][:V_LOL_save])
    data[singleNode][:W_s]= array_to_dict(data[singleNode][:Q], data[singleNode][:W_s_save])
    data[singleNode][:W_d_max]= array_to_dict(data[singleNode][:Q], data[singleNode][:W_d_max_save])
end

In [7]:
#Create dictionary for alle demands dependent on node and load -> array with different times
knownLoads = []
demands = Dict()
defaultPeriods = 1
for singleNode in eachrow(df_demands)
    data_node = Dict(
        :L => string_to_array(string(singleNode.Demands)),
    )    
    if  singleNode.Node ∉ knownLoads
        demands = Dict()
    end
    push!(knownLoads, singleNode.Node)

    demands[singleNode.Loads] = data_node
    data[singleNode.Node][:L] = demands
    
    data[singleNode.Node][:T] = 1 : singleNode.Periods
    if defaultPeriods < singleNode.Periods
        defaultPeriods = singleNode.Periods
    end
end

#Fill periods of Nodes without demands -> Set Default Value
for singleNode in keys(data)
    #if data[singleNode][:T] === nothing
        data[singleNode][:T] = minimum(1):maximum(defaultPeriods)
    #end
end


In [8]:
#create dictionary with all scenarios
scenarios = Dict()
for singleNode in eachrow(df_other)
    data_node = Dict(
        :π => singleNode.Pi,
    )
    scenarios[singleNode.Omega] = data_node
end


#Float muss in Integer umgewandelt werden und Windturbinen speichern
nodes_with_wind_turbines = []#Float muss in Integer umgewandelt werden
for value in keys(data)
    data[value][:Λ] = round.(Int, data[value][:Λ])
    if length(data[value][:Q]) > 0
        push!(nodes_with_wind_turbines, value)
    end
end

#create dictionary with all realized Energy from wind turbine dependent on node, wind turbine, scenario -> array with different times
windenergy = Dict()
data_Omega = Dict()
for singleNode in eachrow(df_realized_energy_scenarios)
    data_node = Dict(
        :W_realized => string_to_array(string(singleNode.Energy)),
    )
    if ! haskey(windenergy, singleNode.Q)
        data_Omega = Dict()
    end
    data_Omega[singleNode.Omega] = data_node
    windenergy[singleNode.Q] = data_Omega
    data[nodes_with_wind_turbines[singleNode.Q]][:Windenergy] = windenergy
end

#Windenergy must be converted into max values in MW
for n in keys(data)
    for q in data[n][:Q]
        for ω in keys(scenarios)
            for t in data[n][:T]
                data[n][:Windenergy][q][ω][:W_realized][t] = data[n][:Windenergy][q][ω][:W_realized][t] * 300
            end
        end
    end
end

LoadError: BoundsError: attempt to access 2-element Vector{Any} at index [3]

### Functions for Variables and Constraints

In [9]:
# Initializes variables
function init_variables(model::JuMP.Model)
    P=@variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #energy generated
    R_U=@variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed upward reserve capacity of generator i
    R_D=@variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed downward reserve capacity of generator i
    r_U=@variable(model, r_U[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #up regulation of generator i in case 𝜔
    r_D=@variable(model, r_D[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #down regulation of generator i in case 𝜔
    L_Shed=@variable(model, L_Shed[n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]] >=0) #loss of load at load demand j in case 𝜔
    W_spill=@variable(model, W_spill[n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]] >=0) #curtailment of turbine q in case 𝜔
    W_s=@variable(model, W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0) #scheduled wind power generation at turbine q
    𝛿=@variable(model, 𝛿[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]])
    𝛿_scenario=@variable(model, 𝛿_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios),t in data[n][:T]]) #voltage angle in case 𝜔
    PF=@variable(model, PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]) #power flow
    PF_scenario=@variable(model, PF_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios), t in data[n][:T]]) #power flow in case 𝜔
    
    
    vars= Dict(
        :P => P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s,
        :𝛿 => 𝛿,
        :𝛿_scenario => 𝛿_scenario,
        :PF => PF,
        :PF_scenario => PF_scenario,
    )    
    return vars
end;

In [10]:
# Initializes constraints
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
    c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
    
    c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
    c3[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_U][n, i, ω, t] <= vars[:R_U][n,i,t]
    c4[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_D][n, i, ω, t] <= vars[:R_D][n,i,t]

    c5[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF][n, ℓ, t] <= data[n][:LC_Max][ℓ]
    c5_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_scenario][n, ℓ, ω, t] <= data[n][:LC_Max][ℓ]
    
    c6[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿][n, ℓ, t] - vars[:𝛿][ℓ, n, t]) * 1 / data[n][:b][ℓ]  == vars[:PF][n, ℓ, t]
    c6_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿_scenario][n, ℓ, ω, t] - vars[:𝛿_scenario][ℓ, n, ω, t]) * 1 / data[n][:b][ℓ] == vars[:PF_scenario][n, ℓ, ω, t]
    
    c9[ω in keys(scenarios), n in keys(data),t in data[n][:T], q in data[n][:Q]], vars[:W_spill][n,q,ω,t] <= data[n][:Windenergy][q][ω][:W_realized][t]
    c10[ω in keys(scenarios), n in keys(data),t in data[n][:T], j in data[n][:J]], vars[:L_Shed][n,j, ω, t] <= data[n][:L][j][:L][t]

    Reference_Node[t in data[1][:T]], vars[:𝛿][1, :, t] .== 0
    Reference_Node_Scenario[ω in keys(scenarios), t in data[1][:T]], vars[:𝛿_scenario][1, :, ω, t] .== 0


    Power_balance_day_ahead[n in keys(data), t in data[n][:T]], 
    sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(vars[:W_s][n,q,t] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0

    Power_balance_at_stage[ω in keys(scenarios), n in keys(data), t in data[n][:T]], 
    sum(vars[:r_U][n,i, ω, t] for i in data[n][:I]) -
    sum(vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
    sum(vars[:L_Shed][n,j, ω, t] for j in data[n][:J]) +
    sum(data[n][:Windenergy][q][ω][:W_realized][t] -vars[:W_s][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[n][:Q]) -
    sum(vars[:PF_scenario][n, ℓ, ω, t] for ℓ in data[n][:Λ]) +
    sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0
    end)
end;

In [11]:
# Initializes objective function
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T]) +
    sum(scenarios[ω][:π] * (sum(data[n][:C_U][i] * vars[:r_U][n,i, ω ,t] - data[n][:C_D][i] * vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
                sum(data[n][:C_cur][q] * vars[:W_spill][n,q, ω, t] for q in data[n][:Q]) +
                sum(data[n][:V_LOL][j] * vars[:L_Shed][n, j, ω, t] for j in data[n][:J])) for n in keys(data), ω in keys(scenarios), t in data[n][:T])
    )
end;

### Model Creation

In [12]:
# Create and print model
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
#print(model)

12 P[2,3.0,1] + 15 R_U[2,3.0,1] + 14 R_D[2,3.0,1] + 32 P[1,1.0,1] + 7 R_U[1,1.0,1] + 5 R_D[1,1.0,1] + 20 P[1,2.0,1] + 11 R_U[1,2.0,1] + 6 R_D[1,2.0,1] + 4.800000000000001 r_U[2,3.0,2,1] - 4.800000000000001 r_D[2,3.0,2,1] + 80 L_Shed[2,2.0,2,1] + 7.199999999999999 r_U[2,3.0,1,1] - 7.199999999999999 r_D[2,3.0,1,1] + 120 L_Shed[2,2.0,1,1] + 12.8 r_U[1,1.0,2,1] - 12.8 r_D[1,1.0,2,1] + 8 r_U[1,2.0,2,1] - 8 r_D[1,2.0,2,1] + 80 L_Shed[1,1.0,2,1] + 19.2 r_U[1,1.0,1,1] - 19.2 r_D[1,1.0,1,1] + 12 r_U[1,2.0,1,1] - 12 r_D[1,2.0,1,1] + 120 L_Shed[1,1.0,1,1]

In [13]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
30 rows, 40 cols, 87 nonzeros
22 rows, 32 cols, 74 nonzeros
22 rows, 32 cols, 74 nonzeros
Presolve : Reductions: rows 22(-25); columns 32(-11); elements 74(-36)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.1999968853e+02 Ph1: 14(21); Du: 9(120) 0s
         19     2.2289837544e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 19
Objective value     :  2.2289837544e+03
HiGHS run time      :          0.00


In [14]:

# Print the values of the variables
println("Variable values:")
println("P:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("P[$n, $i, $t]: ", JuMP.value.(vars[:P][n, i, t]))
end

println("\nR_U:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_U[$n, $i, $t]: ", value(vars[:R_U][n,i, t]))
end

println("\nR_D:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_D[$n, $i, $t]: ", value(vars[:R_D][n,i, t]))end

println("\nL_Shed:")
for n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]
    println("L_Shed[$n, $j, $ω, $t]: ", value(vars[:L_Shed][n,j, ω, t]))
end

println("\nW_spill:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_spill[$n, $q, $ω, $t]: ", value(vars[:W_spill][n,q, ω, t]))
end

println("\nW_s:")
for n in keys(data), q in data[n][:Q], t in data[n][:T]
    println("W_s[$n, $q, $t]: ", value(vars[:W_s][n,q, t]))
end

println("\nPF[n, ℓ, t]:")
for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF[$n, $ℓ, $t]: ", value(vars[:PF][n, ℓ, t]))
end

println("\nPF_scenario[n, ℓ, ω, t]:")
for ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF_scenario[$n, $ℓ, $ω, $t]: ", value(vars[:PF_scenario][n, ℓ, ω, t]))
end

println("\nr_U[n,i, ω, t]:")
for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
    println("r_U[$n, $i, $ω, $t]: ", value(vars[:r_U][n,i, ω, t]))
end

println("\nr_D[n,i, ω, t]:")
for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
    println("r_D[$n, $i, $ω, $t]: ", value(vars[:r_D][n,i, ω, t]))
end

println("\nData values:")

println("\nW_realized:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_realized[$n, $q, $ω, $t]: ", value(data[n][:Windenergy][q][ω][:W_realized][t]))
end

println("\nLoad:")
for n in keys(data), j in data[n][:J], t in data[n][:T]
    println("L[$n, $j, $t]:", value(data[n][:L][j][:L][t]))
end


Variable values:
P:
P[2, 3.0, 1]: 70.0
P[1, 1.0, 1]: -0.0
P[1, 2.0, 1]: 69.46548245774486

R_U:
R_U[2, 3.0, 1]: 0.0
R_U[1, 1.0, 1]: 0.0
R_U[1, 2.0, 1]: -0.0

R_D:
R_D[2, 3.0, 1]: 0.0
R_D[1, 1.0, 1]: 0.0
R_D[1, 2.0, 1]: 0.16294738372661044

L_Shed:
L_Shed[2, 2.0, 2, 1]: 0.0
L_Shed[2, 2.0, 1, 1]: 0.0
L_Shed[1, 1.0, 2, 1]: 0.0
L_Shed[1, 1.0, 1, 1]: 0.0

W_spill:
W_spill[2, 2.0, 2, 1]: 0.0
W_spill[2, 2.0, 1, 1]: 0.0
W_spill[1, 1.0, 2, 1]: 0.0
W_spill[1, 1.0, 1, 1]: 0.0

W_s:
W_s[2, 2.0, 1]: 0.5345175422551429
W_s[1, 1.0, 1]: 0.0

PF[n, ℓ, t]:
PF[2, 1, 1]: 40.534517542255145
PF[1, 2, 1]: -40.534517542255145

PF_scenario[n, ℓ, ω, t]:
PF_scenario[2, 1, 2, 1]: 40.44821340038811
PF_scenario[1, 2, 2, 1]: -40.44821340038811
PF_scenario[2, 1, 1, 1]: 40.39211317882687
PF_scenario[1, 2, 1, 1]: -40.39211317882687

r_U[n,i, ω, t]:
r_U[2, 3.0, 2, 1]: 0.0
r_U[2, 3.0, 1, 1]: -0.0
r_U[1, 1.0, 2, 1]: 0.0
r_U[1, 1.0, 1, 1]: 0.0
r_U[1, 2.0, 2, 1]: 0.0
r_U[1, 2.0, 1, 1]: 0.0

r_D[n,i, ω, t]:
r_D[2, 3.0, 2, 1]

# Robust Optimization

In [15]:
Γ=1.4;

In [16]:
function init_ro_variables_sub(model::Model)
    vars= Dict(
        :λ => @variable(model, λ[n in keys(data), t in data[n][:T]] >=0),
        :λ_ru => @variable(model, λ_ru[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0),
        :λ_rd => @variable(model, λ_rd[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0),
        :λ_l => @variable(model, λ_l[n in keys(data), j in data[n][:J], t in data[n][:T]] >=0),
        :λ_spill => @variable(model, λ_spill[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0),
        :λ_lc => @variable(model, λ_lc[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]] >=0),
        :w_delta => @variable(model, w_delta[n in keys(data), q in data[n][:Q], t in data[n][:T]]),
        :w_delta_up => @variable(model, w_delta_up[n in keys(data), q in data[n][:Q], t in data[n][:T]]),
        :w_delta_down => @variable(model, w_delta_down[n in keys(data), q in data[n][:Q], t in data[n][:T]])
        )
    return vars
end;

In [40]:
function init_ro_constraints_sub(model::Model, data::Dict, vars::Dict)
    cons=Dict(
        :c_ru => @constraint(model, c_ru[n in keys(data), i in data[n][:I], t in data[n][:T]], vars[:λ][n,t] - vars[:λ_ru][n, i, t] <= data[n][:C_U][i]),
        :c_rd => @constraint(model, c_rd[n in keys(data), i in data[n][:I], t in data[n][:T]], -vars[:λ][n,t] - vars[:λ_rd][n, i, t] <= -data[n][:C_D][i]),
        :c_wspill => @constraint(model, c_wspill[n in keys(data), q in data[n][:Q], t in data[n][:T]], -vars[:λ][n,t] -vars[:λ_spill][n, q, t] <= data[n][:C_cur][q]),
        :c_lshed => @constraint(model, c_lshed[n in keys(data), j in data[n][:J], t in data[n][:T]], vars[:λ][n,t] -vars[:λ_l][n, j, t] <= data[n][:V_LOL][j]),
        :c_lc => @constraint(model, c_lc[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], -1/data[n][:b][ℓ]*vars[:λ][n,t] + 1/data[n][:b][ℓ]*vars[:λ][ℓ,t] -1/data[n][:b][ℓ]*vars[:λ_lc][n,ℓ,t] + 1/data[n][:b][ℓ]*vars[:λ_lc][ℓ,n,t]<=0),
            
        :cw1 => @constraint(model, cw1[n in keys(data), q in data[n][:Q], t in data[n][:T]], vars[:w_delta][n, q, t]<= data[n][:W_d_max][q]),
        :cw2 => @constraint(model, cw2[n in keys(data), q in data[n][:Q], t in data[n][:T]], -vars[:w_delta][n, q, t]<= data[n][:W_d_max][q]),
        :cw3 => @constraint(model, cw3[n in keys(data), q in data[n][:Q], t in data[n][:T]], vars[:w_delta][n, q, t] == vars[:w_delta_up][n, q, t]- vars[:w_delta_down][n, q, t]),
        :cw4 => @constraint(model, cw4, sum((vars[:w_delta_up][n, q, t] + vars[:w_delta_down][n, q, t])/ data[n][:W_d_max][q] for n in keys(data), q in data[n][:Q], t in data[n][:T]) <= Γ)
    )
    return cons
end;

In [182]:
function init_ro_objective_sub(model::Model, data::Dict, vars::Dict, vars_master::Dict)
    print(value(vars_ro_master[:𝛿][2,1,1]))
    print(value(vars_ro_master[:𝛿][1,2,1]))
    @objective(model, Max,
    +sum(-sum(vars[:w_delta][n,q,t])*vars[:λ][n,t] for n in keys(data), q in data[n][:Q], t in data[n][:T]) 
    +sum((value(vars_master[:𝛿][n,ℓ,t])- value(vars_master[:𝛿][ℓ,n,t]))* 1/data[n][:b][ℓ]*vars[:λ][n,t] for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T])
    #+sum((value(vars_master[:𝛿][n,ℓ,t])- value(vars_master[:𝛿][ℓ,n,t]))* 1/data[n][:b][ℓ]*vars[:λ][n,t] for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]) 
        
    -sum(value(vars_master[:R_U][n,i,t])* vars[:λ_ru][n,i,t] for n in keys(data), i in data[n][:I], t in data[n][:T])
    -sum(value(vars_master[:R_D][n,i,t])* vars[:λ_rd][n,i,t] for n in keys(data), i in data[n][:I], t in data[n][:T])
    -sum(data[n][:L][j][:L][t]* vars[:λ_l][n,j,t] for n in keys(data), j in data[n][:J], t in data[n][:T])
    -sum((data[n][:W_s][q] + vars[:w_delta][n,q,t]) * vars[:λ_spill][n,q,t] for n in keys(data), q in data[n][:Q], t in data[n][:T])
    -sum(data[n][:LC_Max][ℓ]* vars[:λ_lc][n,ℓ,t] for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T])
    )
end;

In [174]:
function init_ro_variables_master(model::JuMP.Model)

    vars= Dict(
        :alpha=> @variable(model, alpha >= 0),
        :P=> @variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #power generated from generator i
        :R_U=> @variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed upward reserve capacity of generator i
        :R_D=> @variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed downward reserve capacity of generator i
        :𝛿 => @variable(model, 𝛿[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]) #voltage angle
    )
    return vars
end;

In [175]:
function init_ro_constraints_master(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin
        Power_balance_day_ahead[n in keys(data), t in data[n][:T]] , 
        sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(data[n][:W_s][q] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum((vars[:𝛿][n, ℓ, t] - vars[:𝛿][ℓ, n, t])* 1 / data[n][:b][ℓ]  for ℓ in data[n][:Λ])== 0
        c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
        c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
        c3[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿][n,ℓ, t] - vars[:𝛿][ℓ,n, t]) * 1 / data[n][:b][ℓ] <= data[n][:LC_Max][ℓ]
        
        #Reference_Node[t in data[1][:T]], vars[:𝛿][1, :, t] .== 0
    end)
    
end;

In [176]:
function init_ro_objective_master(model::JuMP.Model, data::Dict, vars::Dict)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T]) +
    + vars[:alpha]
    )
end;

In [183]:
model_ro_master=Model(HiGHS.Optimizer)
vars_ro_master=init_ro_variables_master(model_ro_master)
init_ro_constraints_master(model_ro_master, data, vars_ro_master)
init_ro_objective_master(model_ro_master, data, vars_ro_master)
print(model_ro_master)

In [178]:
model_ro_sub=Model(HiGHS.Optimizer)
println(typeof(model_ro_sub))
vars_ro_sub=init_ro_variables_sub(model_ro_sub)
init_ro_objective_sub(model_ro_sub, data, vars_ro_sub, vars_ro_master)
print(model_ro_sub)

Model


┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP C:\Users\lukas\.julia\packages\JuMP\HjlGr\src\optimizer_interface.jl:695


LoadError: OptimizeNotCalled()

In [179]:
function benders_decomp(master::JuMP.Model, sub::JuMP.Model, data::Dict, vars_ro_sub, vars_ro_master)
    #initialize master model variables
    optimize!(master)
    
    #init sub cons and objective
    cons=init_ro_constraints_sub(model_ro_sub, data, vars_ro_sub, vars_ro_master)
    #init_ro_objective_sub(model_ro_sub, data, vars_ro_sub, vars_ro_master)
    
    #initalize variables
    UB_master=0
    for n in keys(data)
    for i in data[n][:I]
        for t in data[n][:T]
            # Update UP_master using the defined n, i, and t
            UB_master += value(vars_ro_master[:P][n, i, t]) * data[n][:C][i] +
                         value(vars_ro_master[:R_U][n, i, t]) * data[n][:C_RU][i] +
                         value(vars_ro_master[:R_D][n, i, t]) * data[n][:C_RD][i]
            end
        end
    end
    print(UB_master)
    UB=0 #intial value for the cancel condition
    LB=-1
    i=1; #number of iteration (just for printing purposes)
    
    #solving the problem with Benders decomposition using dual dynamic programming – DDP

    while UB!=LB && i<15
        print("\n iteration no $i \n")

        # optimize dual subproblem with the fixed values of the masterproblem
        print("\n model sub\n")
        print(value(vars_ro_master[:𝛿][2,1,1]))
        print(value(vars_ro_master[:𝛿][1,2,1]))
        print("\n")
        init_ro_objective_sub(sub, data, vars_ro_sub, vars_ro_master)
        print("\n model sub\n")
        print(sub)
        print("\n solve sub start \n")
        optimize!(sub)
        print("\n solve sub end \n")
        UB_master=0
        for n in keys(data)
            for i in data[n][:I]
                    for t in data[n][:T]
                        # Update UP_master using the defined n, i, and t
                        UB_master += value(vars_ro_master[:P][n, i, t]) * data[n][:C][i] +
                                     value(vars_ro_master[:R_U][n, i, t]) * data[n][:C_RU][i] +
                                     value(vars_ro_master[:R_D][n, i, t]) * data[n][:C_RD][i]
                    end
            end
        end
        UB =UB_master + objective_value(sub)
        # add benders cut (as constraint with dual variable from the subproblem)
        @constraint(master, 
                +sum(-sum(value(vars_ro_sub[:w_delta][n,q,t]))*value(vars_ro_sub[:λ][n,t]) for n in keys(data), q in data[n][:Q], t in data[n][:T]) 
                +sum(-sum(vars_ro_master[:𝛿][n,ℓ,t]* 1/data[n][:b][ℓ])*value(vars_ro_sub[:λ][n,t]) for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]) 
                +sum(sum(vars_ro_master[:𝛿][ℓ,n,t]* 1/data[n][:b][ℓ])*value(vars_ro_sub[:λ][n,t]) for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T])  
                -sum(vars_ro_master[:R_U][n,i,t]* value(vars_ro_sub[:λ_ru][n,i,t]) for n in keys(data), i in data[n][:I], t in data[n][:T])
                -sum(vars_ro_master[:R_D][n,i,t]* value(vars_ro_sub[:λ_rd][n,i,t]) for n in keys(data), i in data[n][:I], t in data[n][:T])
                -sum(data[n][:L][j][:L][t]* value(vars_ro_sub[:λ_l][n,j,t]) for n in keys(data), j in data[n][:J], t in data[n][:T])
                -sum((data[n][:W_s][q] + value(vars_ro_sub[:w_delta][n,q,t])) * value(vars_ro_sub[:λ_spill][n,q,t]) for n in keys(data), q in data[n][:Q], t in data[n][:T])
                -sum(data[n][:LC_Max][ℓ]* value(vars_ro_sub[:λ_lc][n,ℓ,t]) for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]) <= vars_ro_master[:alpha])

        # solve masterproblem with new constraint
        optimize!(master)
        LB=objective_value(master)
        print("LB:\n")
        print(LB)
        print("\nUP:\n")
        print(UB)
        i=i+1
    end
    
end;

In [180]:
benders_decomp(model_ro_master, model_ro_sub, data, vars_ro_sub, vars_ro_master)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10 rows, 5 cols, 17 nonzeros
1 rows, 3 cols, 3 nonzeros
1 rows, 3 cols, 3 nonzeros
Presolve : Reductions: rows 1(-9); columns 3(-9); elements 3(-20)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.9044601075e-03 Pr: 1(95) 0s
          1     1.3800000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  1.3800000000e+03
HiGHS run time      :          0.00


LoadError: MethodError: no method matching init_ro_constraints_sub(::Model, ::Dict{Any, Any}, ::Dict{Symbol, JuMP.Containers.SparseAxisArray{VariableRef, N} where N}, ::Dict{Symbol, Any})

[0mClosest candidates are:
[0m  init_ro_constraints_sub(::Model, ::Dict, ::Dict)
[0m[90m   @[39m [36mMain[39m [90m[4mIn[40]:1[24m[39m


In [149]:
print(model_ro_sub)

In [70]:
# Print the values of the variables
println("Variable values:")
print("alpha:")
print(value(vars_ro_master[:alpha]))
println("\nP:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("P[$n, $i, $t]: ", JuMP.value.(vars_ro_master[:P][n, i, t]))
end

println("\nR_U:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_U[$n, $i, $t]: ", value(vars_ro_master[:R_U][n,i, t]))
end

println("\nR_D:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_D[$n, $i, $t]: ", value(vars_ro_master[:R_D][n,i, t]))end

println("\n𝛿:")
for n in keys(data),ℓ in data[n][:Λ], t in data[n][:T]
    println("𝛿[$n,$ℓ $t]: ", value(vars_ro_master[:𝛿][n,ℓ, t]))
end

Variable values:
alpha:0.0
P:
P[2, 3.0, 1]: 65.0
P[1, 1.0, 1]: 0.0
P[1, 2.0, 1]: 30.0

R_U:
R_U[2, 3.0, 1]: 0.0
R_U[1, 1.0, 1]: 0.0
R_U[1, 2.0, 1]: 0.0

R_D:
R_D[2, 3.0, 1]: 0.0
R_D[1, 1.0, 1]: 0.0
R_D[1, 2.0, 1]: 0.0

𝛿:
𝛿[2,1 1]: 0.0
𝛿[1,2 1]: -7.800000000000001


In [28]:
# Print the values of the variables
println("Variable values:")

# println("\n:λ:")
# for n in keys(data), t in data[n][:T]
#     println(":λ[$n, $t]: ", JuMP.value.(vars_ro_sub[:λ][n, t]))
# end

Variable values:


# Benders Decomposition with Stochastic programming

In [29]:
function init_vars_master(model::Model)
    vars= Dict(
        :P => @variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #energy generated,
        :R_U => @variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed upward reserve capacity of generator i,
        :R_D => @variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed downward reserve capacity of generator i,
        :W_s => @variable(model, W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0), #scheduled wind power generation at turbine q,
        :𝛿 => @variable(model, 𝛿[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]), #voltage angle,
        :PF => @variable(model, PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]), #power flow,
        :alpha => @variable(model, alpha >=0)
    )
    return vars
end;

In [30]:
function init_vars_sub(model::Model)
    vars= Dict(
        :r_U => @variable(model, r_U[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0), #up regulation of generator i in case 𝜔
        :r_D => @variable(model, r_D[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0), #down regulation of generator i in case 𝜔
        :L_Shed => @variable(model, L_Shed[n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]] >=0), #loss of load at load demand j in case 𝜔
        :W_spill => @variable(model, W_spill[n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]] >=0), #curtailment of turbine q in case 𝜔
        :𝛿_scenario => @variable(model, 𝛿_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios),t in data[n][:T]]), #voltage angle in case 𝜔
        :PF_scenario => @variable(model, PF_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios), t in data[n][:T]]), #power flow in case 𝜔
        
        :W_s_sub => @variable(model, W_s_sub[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0), #scheduled wind power generation at turbine q
        :PF_sub => @variable(model, PF_sub[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]), #power flow
        :R_U_sub => @variable(model, R_U_sub[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed upward reserve capacity of generator i,
        :R_D_sub => @variable(model, R_D_sub[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed downward reserve capacity of generator i,
    )
    return vars
end;

In [31]:
# Initializes constraints
function init_cons_master(model::Model, data::Dict, vars::Dict)
    @constraints(model, begin 
    c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
    
    c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
    c5[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF][n, ℓ, t] <= data[n][:LC_Max][ℓ]
    
    c6[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿][n,ℓ, t] - vars[:𝛿][ℓ,n, t]) * 1 / data[n][:b][ℓ]  == vars[:PF][n, ℓ, t]
       
    Reference_Node[t in data[1][:T]], vars[:𝛿][1, :, t] .== 0
            
    Power_balance_day_ahead[n in keys(data), t in data[n][:T]], 
    sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(vars[:W_s][n,q,t] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0

    end)
end;

In [32]:
# Initializes constraints
function init_cons_sub(model::Model, data::Dict, vars::Dict, vars_master::Dict)
    cons= Dict(

    :c3 => @constraint(model,c3[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_U][n, i, ω, t] <= vars[:R_U_sub][n,i,t]),
    :c4 => @constraint(model,c4[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_D][n, i, ω, t] <= vars[:R_D_sub][n,i,t]),
    :c5_scenario => @constraint(model,c5_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_scenario][n, ℓ, ω, t] <= data[n][:LC_Max][ℓ]),
    
    :c6_scednario => @constraint(model,c6_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿_scenario][n, ℓ, ω, t] - vars[:𝛿_scenario][ℓ,n, ω, t]) * 1 / data[n][:b][ℓ] == vars[:PF_scenario][n, ℓ, ω, t]),
       
    
    :c9 => @constraint(model,c9[ω in keys(scenarios), n in keys(data),t in data[n][:T], q in data[n][:Q]], vars[:W_spill][n,q,ω,t] <= data[n][:Windenergy][q][ω][:W_realized][t]),
    :c10 => @constraint(model,c10[ω in keys(scenarios), n in keys(data),t in data[n][:T], j in data[n][:J]], vars[:L_Shed][n,j, ω, t] <= data[n][:L][j][:L][t]),
    
    :c11 => @constraint(model, Reference_Node_Scenario[ω in keys(scenarios), t in data[1][:T]], vars[:𝛿_scenario][1, :, ω, t] .== 0),

    :Power_balance_at_stage => @constraint(model,Power_balance_at_stage[ω in keys(scenarios), n in keys(data), t in data[n][:T]], 
    sum(vars[:r_U][n,i, ω, t] for i in data[n][:I]) -
    sum(vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
    sum(vars[:L_Shed][n,j, ω, t] for j in data[n][:J]) +
    sum(data[n][:Windenergy][q][ω][:W_realized][t] -vars[:W_s_sub][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[n][:Q]) -
    sum(vars[:PF_scenario][n, ℓ, ω, t] for ℓ in data[n][:Λ]) +
    sum(vars[:PF_sub][n,ℓ,t] for ℓ in data[n][:Λ]) == 0),
    
    :η_RU => @constraint(model,η_RU[n in keys(data), i in data[n][:I], t in data[n][:T]], vars[:R_U_sub][n,i,t] == value(vars_master[:R_U][n,i,t])),
    :η_RD => @constraint(model,η_RD[n in keys(data), i in data[n][:I], t in data[n][:T]], vars[:R_D_sub][n,i,t] == value(vars_master[:R_D][n,i,t])),
    :η_W_s => @constraint(model,η_W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]], vars[:W_s_sub][n,q,t] == value(vars_master[:W_s][n,q,t])),
    :η_PF => @constraint(model,η_PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_sub][n,ℓ,t] == value(vars_master[:PF][n, ℓ, t]))
    )
end;

In [33]:
# Initializes objective function master
function init_obj_master(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T])
    + vars[:alpha]
    )
end;

In [34]:
# Initializes objective function sub
function init_obj_sub(model, data, vars)
    @objective(model, Min, sum(scenarios[ω][:π] * (sum(data[n][:C_U][i] * vars[:r_U][n,i, ω ,t] - data[n][:C_D][i] * vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
                sum(data[n][:C_cur][q] * vars[:W_spill][n,q, ω, t] for q in data[n][:Q]) +
                sum(data[n][:V_LOL][j] * vars[:L_Shed][n, j, ω, t] for j in data[n][:J])) for n in keys(data), ω in keys(scenarios), t in data[n][:T])
    )
end;

bd_master=Model(HiGHS.Optimizer)
vars_master=init_vars_master(bd_master)
init_cons_master(bd_master, data, vars_master)
init_obj_master(bd_master, data, vars_master)
print(bd_master)
#optimize!(bd_master)

#print(value(vars_master[:W_s][1,1.0,1]))
bd_sub=Model(GLPK.Optimizer)
#bd_sub=Model(HiGHS.Optimizer)
vars_sub=init_vars_sub(bd_sub)
#cons_sub=init_cons_sub(bd_sub, data, vars_sub, vars_master)
#init_obj_sub(bd_sub, data, vars_sub)
#optimize!(bd_sub)
#print(objective_value(bd_sub))
print(bd_sub)

In [35]:
#optimize!(bd_sub)
#obj_value = objective_value(bd_sub)
#print(obj_value)
#for n in keys(data)
#    for q in data[n][:Q]
#        for t in data[n][:T]
#            # Update UP_master using the defined n, i, and t
#            print(dual(cons_sub[:η_W_s][n,q,t]))
#        end
#    end
#end

In [36]:
function benders_decomp_st(master::Model, data::Dict, vars_master::Dict)
    #initialize master model variables
    optimize!(master)
    
    #init sub cons and objective
    
    #initalize variables
    UB_master=0
    for n in keys(data)
    for i in data[n][:I]
        for t in data[n][:T]
            # Update UP_master using the defined n, i, and t
            UB_master += value(vars_master[:P][n, i, t]) * data[n][:C][i] +
                         value(vars_master[:R_U][n, i, t]) * data[n][:C_RU][i] +
                         value(vars_master[:R_D][n, i, t]) * data[n][:C_RD][i]
            end
        end
    end
    print(UB_master)
    UB=0 #intial value for the cancel condition
    LB=-1
    i=1; #number of iteration (just for printing purposes)
    
    #solving the problem with Benders decomposition using dual dynamic programming – DDP

    while UB!=LB && i<9
        print("\n iteration no $i \n")
        
        sub=Model(GLPK.Optimizer)
        vars_sub=init_vars_sub(sub)
        cons_sub=init_cons_sub(sub, data, vars_sub, vars_master)
        init_obj_sub(sub, data, vars_sub)

        # optimize dual subproblem with the fixed values of the masterproblem
        optimize!(sub)
        obj_sub=objective_value(sub)
        print("\n obj_sub \n")
        #print(sub)
        print(obj_sub)
        # Print the values of the variables
        


#         println("\nL_Shed:")
#         for n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]
#             println("L_Shed[$n, $j, $ω, $t]: ", value(vars_sub[:L_Shed][n,j, ω, t]))
#         end

#         println("\nW_spill:")
#         for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
#             println("W_spill[$n, $q, $ω, $t]: ", value(vars_sub[:W_spill][n,q, ω, t]))
#         end

#         println("\nPF_scenario[n, ℓ, ω, t]:")
#         for ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
#             println("PF_scenario[$n, $ℓ, $ω, $t]: ", value(vars_sub[:PF_scenario][n, ℓ, ω, t]))
#         end

#         println("\nr_U[n,i, ω, t]:")
#         for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
#             println("r_U[$n, $i, $ω, $t]: ", value(vars_sub[:r_U][n,i, ω, t]))
#         end

#         println("\nr_D[n,i, ω, t]:")
#         for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
#             println("r_D[$n, $i, $ω, $t]: ", value(vars_sub[:r_D][n,i, ω, t]))
#         end

        
        # add benders cut (as constraint with dual variable from the subproblem)
        @constraint(master, obj_sub
            +sum(dual(cons_sub[:η_W_s][n,q,t])*(vars_master[:W_s][n, q, t]- value(vars_master[:W_s][n, q, t])) for n in keys(data), q in data[n][:Q], t in data[n][:T]) 
            + sum(dual(cons_sub[:η_RD][n,i,t])* (vars_master[:R_D][n, i, t]- value(vars_master[:R_D][n, i, t])) for n in keys(data), i in data[n][:I], t in data[n][:T]) 
            + sum(dual(cons_sub[:η_RU][n,i,t])* (vars_master[:R_U][n, i, t]- value(vars_master[:R_U][n, i, t])) for n in keys(data), i in data[n][:I], t in data[n][:T])
            + sum(dual(cons_sub[:η_PF][n,ℓ,t])* (vars_master[:PF][n, ℓ, t]- value(vars_master[:PF][n, ℓ, t])) for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T])<= vars_master[:alpha]
        )
        #print(master)
        # solve masterproblem with new constraint
        optimize!(master)
        LB=objective_value(master)
        UB=0
        for n in keys(data)
            for i in data[n][:I]
                    for t in data[n][:T]
                        # Update UP_master using the defined n, i, and t
                        UB += value(vars_master[:P][n, i, t]) * data[n][:C][i] +
                                     value(vars_master[:R_U][n, i, t]) * data[n][:C_RU][i] +
                                     value(vars_master[:R_D][n, i, t]) * data[n][:C_RD][i]
                    end
            end
        end
        UB =UB + objective_value(sub)
        print(UB)
        print("LB:\n")
        print(LB)
        print("\nUP:\n")
        print(UB)
        i=i+1
    end
    
end;

In [37]:
bd_master=Model(HiGHS.Optimizer)
vars_master=init_vars_master(bd_master)
init_cons_master(bd_master, data, vars_master)
init_obj_master(bd_master, data, vars_master)

bd_sub=Model(GLPK.Optimizer)
vars_sub=init_vars_sub(bd_sub)
#print(bd_sub)

benders_decomp_st(bd_master, data, vars_master)



Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10 rows, 8 cols, 17 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-13); columns 0(-16); elements 0(-28) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
0.0
 iteration no 1 

 obj_sub 
27880.06070085084Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.7641188137e-03 Pr: 1(108.906) 0s
          2     4.1166919587e+02 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  4.1166919587e+02
HiGHS run time      :          0.00
28291.729896719626LB:
411.669195868785
UP:
28291.729896719626
 iteration no 2 

 obj_sub 
18000.000000000004Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  I

In [38]:
cons_sub=init_cons_sub(bd_sub, data, vars_sub, vars_master)
init_obj_sub(bd_sub, data, vars_sub)
#print(bd_sub)

4.800000000000001 r_U[2,3.0,2,1] - 4.800000000000001 r_D[2,3.0,2,1] + 80 L_Shed[2,2.0,2,1] + 7.199999999999999 r_U[2,3.0,1,1] - 7.199999999999999 r_D[2,3.0,1,1] + 120 L_Shed[2,2.0,1,1] + 12.8 r_U[1,1.0,2,1] - 12.8 r_D[1,1.0,2,1] + 8 r_U[1,2.0,2,1] - 8 r_D[1,2.0,2,1] + 80 L_Shed[1,1.0,2,1] + 19.2 r_U[1,1.0,1,1] - 19.2 r_D[1,1.0,1,1] + 12 r_U[1,2.0,1,1] - 12 r_D[1,2.0,1,1] + 120 L_Shed[1,1.0,1,1]

In [39]:
print(bd_master)

# Print the values of the variables
println("Variable values:")
println("P:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("P[$n, $i, $t]: ", JuMP.value.(vars_master[:P][n, i, t]))
end

println("\nR_U:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_U[$n, $i, $t]: ", value(vars_master[:R_U][n,i, t]))
end

println("\nR_D:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_D[$n, $i, $t]: ", value(vars_master[:R_D][n,i, t]))end


println("\nW_s:")
for n in keys(data), q in data[n][:Q], t in data[n][:T]
    println("W_s[$n, $q, $t]: ", value(vars_master[:W_s][n,q, t]))
end

println("\nPF[n, ℓ, t]:")
for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF[$n, $ℓ, $t]: ", value(vars_master[:PF][n, ℓ, t]))
end

println("\nData values:")

println("\nW_realized:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_realized[$n, $q, $ω, $t]: ", value(data[n][:Windenergy][q][ω][:W_realized][t]))
end

println("\nLoad:")
for n in keys(data), j in data[n][:J], t in data[n][:T]
    println("L[$n, $j, $t]:", value(data[n][:L][j][:L][t]))
end

Variable values:
P:
P[2, 3.0, 1]: 70.0
P[1, 1.0, 1]: 0.0
P[1, 2.0, 1]: 69.46548245774486

R_U:
R_U[2, 3.0, 1]: 0.0
R_U[1, 1.0, 1]: -0.0
R_U[1, 2.0, 1]: 0.0

R_D:
R_D[2, 3.0, 1]: 0.0
R_D[1, 1.0, 1]: -0.0
R_D[1, 2.0, 1]: -0.0

W_s:
W_s[2, 2.0, 1]: 0.0
W_s[1, 1.0, 1]: 0.5345175422551449

PF[n, ℓ, t]:
PF[2, 1, 1]: 40.0
PF[1, 2, 1]: -40.0

Data values:

W_realized:
W_realized[2, 2.0, 2, 1]: 0.44821340038811286
W_realized[2, 2.0, 1, 1]: 0.3921131788268655
W_realized[1, 1.0, 2, 1]: 0.24925152559364394
W_realized[1, 1.0, 1, 1]: 0.1424043634282774

Load:
L[2, 2.0, 1]:30.0
L[1, 1.0, 1]:110.0
